### Queries 
Quries the FPL database & gets the data for the ML training.This class does the reading/querying from the database.

### Operations
- Gets player statistics from gameweeks
- Gets the dataset for ML training
- Calculates the recent form for all players

### References
https://docs.python.org/3/library/sqlite3.html

https://fantasy.premierleague.com/api/bootstrap-static/

https://pandas.pydata.org/docs/reference/api/pandas.read_sql_query.html

In [18]:
# Querrying Data from database
# importing sqlite to connect to the database
# importing pandas for data manipulation
import sqlite3
import pandas as pd

class QueryingData:
    """
    Queries FPL data from the database
    
    Retrieves player statistics & gameweek data for ML training
    """

    # using ../ as database is not in this folder
    def __init__(self, db_path='../fpl_data.db'):
        """
        Creating the query class with path to the database
        
        SQLite database file 'fpl_data.db'
        """
        # Storing the database location
        self.db_path = db_path

    # Get a specific players statistics
    def getPlayerStats(self, player_name):
        """
        Getting stats for a certain player across all gameweeks
        
        Search for player name & returns players gameweek data
            
        """
        # Connecting to the database
        conn = sqlite3.connect(self.db_path)
        
        # Using sql query to get player stats across from all the gameweeks
        # Uses LIKE for partial name matching
        query = '''
            SELECT p.name, g.gameweek, g.minutes, g.goals_scored, 
                   g.assists, g.total_points
            FROM gameweek_data g
            JOIN players p ON g.player_id = p.player_id
            WHERE p.name LIKE ?
            ORDER BY g.gameweek
        '''
        
        # Calling the query with parameter
        df = pd.read_sql_query(query, conn, params=(f'%{player_name}%',))
        
        # Closing the connection
        conn.close()
        return df